In [1]:
!jupyter nbconvert --to script Functions.ipynb
import Functions
from SmartApi import SmartConnect 
import pyotp
from logzero import logger
import matplotlib.pyplot as plt

api_key = ''
username = ''
pwd = ''
smartApi = SmartConnect(api_key)
try:
    token = ""
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

data = smartApi.generateSession(username, pwd, totp)

if data['status'] == False:
    logger.error(data)
    
else:
    # login api call
    # logger.info(f"You Credentials: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    # fetch the feedtoken
    feedToken = smartApi.getfeedToken()
    # fetch User Profile
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']


[NbConvertApp] Converting notebook Functions.ipynb to script
[NbConvertApp] Writing 7594 bytes to Functions.py


In [ ]:
import pandas as pd
data_table = pd.DataFrame(data)
data_table
data_table.to_csv('data_table.csv')


In [ ]:
data_table['data']['feedToken']

In [ ]:
#place a order
try:
        orderparams = {
            "variety": "NORMAL",
            "tradingsymbol": "SBIN-EQ",
            "symboltoken": "3045",
            "transactiontype": "BUY",
            "exchange": "NSE",
            "ordertype": "LIMIT",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "19500",
            "squareoff": "0",
            "stoploss": "0",
            "quantity": "1"
            }

        response = smartApi.placeOrderFullResponse(orderparams)
        logger.info(f"PlaceOrder : {response}")
except Exception as e:
        logger.exception(f"Order placement failed: {e}")

response_table = pd.DataFrame(response)
response_table

In [ ]:
#instrument data
import requests
import json 
import pandas as pd

#to fetch any intrument data follow up "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json" link

response = requests.get("https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json")
instrument_details = response.json()
instrument_details

instrument_details_table = pd.DataFrame(instrument_details)
instrument_details_table.to_csv('intrument_details.csv', index = False)
print('import sucessfull')



# Historic Data

In [2]:
try:
        historicParam= {
            "exchange": "NSE",
            "symboltoken": "6705", 
            "interval": "FIVE_MINUTE",
            "fromdate": '2024-01-01 15:30',
            "todate": '2024-02-08 15:30'
            }

        candel_data = smartApi.getCandleData(historicParam)
except Exception as e:
        logger.exception(f"Historic Api failed: {e}")
    
candel_data

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': [['2024-01-02T09:15:00+05:30', 648.0, 648.95, 644.8, 647.45, 131861],
  ['2024-01-02T09:20:00+05:30', 647.5, 647.9, 645.35, 645.75, 72175],
  ['2024-01-02T09:25:00+05:30', 645.9, 652.0, 645.0, 650.95, 193354],
  ['2024-01-02T09:30:00+05:30', 651.0, 652.0, 649.25, 652.0, 179254],
  ['2024-01-02T09:35:00+05:30', 651.6, 652.55, 650.2, 651.8, 126912],
  ['2024-01-02T09:40:00+05:30', 651.8, 653.15, 651.3, 652.95, 63136],
  ['2024-01-02T09:45:00+05:30', 653.0, 653.0, 651.25, 652.2, 43265],
  ['2024-01-02T09:50:00+05:30', 652.2, 652.25, 651.8, 652.05, 47899],
  ['2024-01-02T09:55:00+05:30', 652.05, 652.15, 646.05, 646.75, 168582],
  ['2024-01-02T10:00:00+05:30', 646.7, 647.15, 643.3, 645.85, 126761],
  ['2024-01-02T10:05:00+05:30', 645.95, 647.95, 645.95, 647.75, 31529],
  ['2024-01-02T10:10:00+05:30', 647.6, 648.5, 646.4, 647.9, 35457],
  ['2024-01-02T10:15:00+05:30', 647.8, 648.5, 647.6, 648.0, 29936],
  ['2024-01-02T10:20:0

In [3]:
import pandas as pd

df = pd.DataFrame(candel_data['data'], columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume'])

date_pattern = r'(\d{4}-\d{2}-\d{2})'
df['date'] = df['timestamp'].str.extract(date_pattern)

df['timestamp']=pd.to_datetime(df['timestamp'])
df['time'] = df['timestamp'].dt.time

df

,timestamp,open,high,low,close,volume,date,time
0,2024-01-02 09:15:00+05:30,648.0,648.95,644.80,647.45,131861,2024-01-02,09:15:00
1,2024-01-02 09:20:00+05:30,647.5,647.90,645.35,645.75,72175,2024-01-02,09:20:00
2,2024-01-02 09:25:00+05:30,645.9,652.00,645.00,650.95,193354,2024-01-02,09:25:00
3,2024-01-02 09:30:00+05:30,651.0,652.00,649.25,652.00,179254,2024-01-02,09:30:00
4,2024-01-02 09:35:00+05:30,651.6,652.55,650.20,651.80,126912,2024-01-02,09:35:00
...,...,...,...,...,...,...,...,...
1977,2024-02-08 11:30:00+05:30,463.7,464.65,462.70,464.40,62990,2024-02-08,11:30:00
1978,2024-02-08 11:35:00+05:30,464.1,465.65,463.95,465.65,60883,2024-02-08,11:35:00
1979,2024-02-08 11:40:00+05:30,465.7,465.80,464.55,464.85,57010,2024-02-08,11:40:00
1980,2024-02-08 11:45:00+05:30,464.6,465.40,464.10,465.10,45089,2024-02-08,11:45:00


# SMA

In [4]:
sma = Functions.calculate_sma(df['close'], 5)
type(sma)

list

# RMA

In [5]:
rma = Functions.calculate_rma(sma[4:], 5)
rma

#if putting sma as a parameter make sure to slice out the none values

[649.5899999999999,
 650.47,
 651.678,
 652.0956000000001,
 651.33912,
 650.235824,
 649.1831648,
 648.2846329600001,
 647.4569265920001,
 647.3633853184,
 647.12867706368,
 646.7297354127361,
 645.9459470825473,
 645.0611894165095,
 644.6842378833019,
 644.9368475766604,
 644.8753695153321,
 645.0710739030666,
 645.2862147806132,
 645.0172429561227,
 644.4834485912246,
 644.2486897182449,
 644.2177379436489,
 644.1235475887299,
 644.248709517746,
 644.5137419035492,
 644.5747483807099,
 644.602949676142,
 644.8165899352284,
 644.9873179870457,
 645.0934635974091,
 645.7946927194819,
 646.4469385438964,
 646.8733877087793,
 647.2946775417558,
 647.8029355083512,
 647.9285871016702,
 647.9937174203341,
 648.2067434840669,
 648.3133486968134,
 648.2866697393627,
 648.0813339478725,
 647.9042667895744,
 647.7568533579149,
 647.6713706715828,
 647.5742741343165,
 647.6348548268634,
 647.6629709653728,
 647.7165941930747,
 647.807318838615,
 647.8174637677231,
 647.7874927535447,
 647.66949

# EMA

In [6]:
ema = Functions.calculate_ema(df['close'], 5)
ema

[647.45,
 646.8833333333334,
 648.238888888889,
 649.4925925925927,
 650.2617283950618,
 651.1578189300412,
 651.5052126200276,
 651.6868084133517,
 650.0412056089012,
 648.6441370726009,
 648.3460913817339,
 648.1973942544894,
 648.1315961696596,
 647.8210641131064,
 646.714042742071,
 646.326028494714,
 645.384018996476,
 644.7393459976507,
 645.1428973317672,
 645.6119315545116,
 645.3579543696744,
 645.171969579783,
 644.964646386522,
 644.6430975910147,
 644.2787317273431,
 644.2024878182287,
 644.4349918788191,
 644.2899945858794,
 644.4933297239197,
 644.6788864826132,
 644.4859243217422,
 644.6572828811616,
 644.8715219207744,
 645.1476812805163,
 645.2651208536776,
 646.2934139024518,
 646.9289426016346,
 647.0026284010899,
 647.2684189340599,
 647.5622792893734,
 647.8748528595822,
 648.0665685730548,
 648.1777123820366,
 648.1851415880244,
 648.0734277253496,
 647.7989518168997,
 647.7326345445999,
 647.7217563630667,
 647.7645042420445,
 647.626336161363,
 647.6675574409087

# RSI

In [7]:
rsi = Functions.calculate_rsi(df['close'], 5)
rsi
#problem

[0,
 0,
 93.86281588447643,
 96.84893419833165,
 66.18112729575162,
 96.65287702143617,
 24.542806665712988,
 14.056005250489463,
 0.18376933063871093,
 0.099985708916833,
 82.81278443332947,
 87.04635511717596,
 92.88679354896003,
 4.879818182423847,
 0.2872429471816673,
 64.76565747218929,
 8.85689837386775,
 8.01330546518119,
 96.29261523152284,
 98.27694596521953,
 11.450814484556659,
 10.134144381734444,
 2.6155188345334466,
 0.2855054260623433,
 0.06147367494757816,
 81.35252483395521,
 97.6437326768059,
 17.35838962781908,
 83.8310696520597,
 90.3196906495212,
 6.58888323332188,
 82.67368625267301,
 92.65063676865665,
 98.48117354170557,
 33.01029136323784,
 98.61498316193637,
 78.40792191647807,
 9.594936457318596,
 75.6744643147011,
 91.80373660925801,
 98.10064881177342,
 63.341349771316445,
 22.853596096799407,
 1.6579813928160263,
 0.18189182826975525,
 0.021074040723618737,
 72.06309948069537,
 86.23383476800146,
 97.13461931806759,
 6.841643474600843,
 80.25578723112702,


# MFI

In [8]:
mfi = Functions.calculate_mfi(df['high'], df['low'], df['close'], df['volume'], 5)
mfi

[None,
 None,
 None,
 None,
 68.66791601392617,
 71.87777275116768,
 52.62820864462043,
 14.077643789710066,
 14.119599468798853,
 7.530941079931168,
 16.321566088460344,
 24.723530940863768,
 38.36756239260265,
 50.48278329431936,
 52.15499052281979,
 29.748357638750946,
 13.447400019335376,
 23.9798286565927,
 37.92574581009821,
 20.732750380576448,
 24.594921297430147,
 30.280423350315687,
 14.750624589084225,
 0.0,
 19.58041262016613,
 30.308699114533297,
 41.60476701113812,
 51.39487791108802,
 82.47880246275565,
 49.85285030513632,
 40.023978062008084,
 63.33492605785075,
 67.23046133131737,
 47.9432206606412,
 91.25642094872529,
 39.62836195562023,
 32.45607008692761,
 35.64170644642205,
 46.60742434346366,
 37.7512480762719,
 64.27882917127906,
 75.3962138204084,
 59.80926847332141,
 28.719795895323344,
 0.0,
 16.049699025458324,
 33.27565846104332,
 51.62993888650279,
 52.892212242619294,
 69.96409559561857,
 55.57709160850415,
 50.374251447505394,
 49.92638782619081,
 52.7830

# HMA

In [9]:
hma = Functions.calculate_hma(df['close'], 5)
hma

array([650.10333333, 650.59111111, 651.98888889, ..., 463.95333333,
       464.81888889, 465.02777778])

# VWAP

In [10]:
vwap = Functions.calculate_vwap(df['close'], 5)
vwap

array([6.4745000e+02, 1.2932000e+03, 1.9441500e+03, ..., 1.3362157e+06,
       1.3366808e+06, 1.3371467e+06])

# using the df for plotting candleticks

In [11]:
import plotly.graph_objects as go
import plotly

fig = go.Figure(data=[go.Candlestick(
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='instrument'
)])

# Create the scatter trace using go.Scatter() with parentheses
sma_plot = go.Scatter(y = sma, name = 'sma')
rma_plot = go.Scatter(y = rma, name = 'rma')
ema_plot = go.Scatter(y = ema, name = 'ema')
rsi_plot = go.Scatter(y = rsi, name = 'rsi') 
mfi_plot = go.Scatter(y = mfi, name = 'mfi')
hma_plot = go.Scatter(y = hma, name = 'hma')
# vwap_plot = go.Scatter(y = vwap, name = 'vwap') #problem

fig.add_trace(sma_plot)
fig.add_trace(rma_plot)
fig.add_trace(ema_plot)
fig.add_trace(rsi_plot)
fig.add_trace(mfi_plot)
fig.add_trace(hma_plot)
# fig.add_trace(vwap_plot)

fig.add_shape(
    type='line',
    x0=df.index.min(),
    x1=df.index.max(),
    y0=50,
    y1=50,
    line=dict(color='black', width=1, dash='dash')
)
fig.add_shape(
    type='line',
    x0=df.index.min(),
    x1=df.index.max(),
    y0=30,
    y1=30,
    line=dict(color='black', width=1, dash='dash')
)
fig.add_shape(
    type='line',
    x0=df.index.min(),
    x1=df.index.max(),
    y0=70,
    y1=70,
    line=dict(color='black', width=1, dash='dash')
)

fig.update_layout(xaxis_rangeslider_visible=False)
plotly.offline.plot(fig)

# fig.show() 
#to show in the console


'temp-plot.html'

# websocket realtime data

In [ ]:
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
from logzero import logger
from websocket import WebSocketApp
import io
import logging


AUTH_TOKEN = ""
API_KEY = ''
CLIENT_CODE = ''
FEED_TOKEN = ""
correlation_id = ""
action = 1
mode = 2
token_list = [
    {
        "exchangeType": 1,
        "tokens": ["15083"]
    }
]



sws = SmartWebSocketV2(AUTH_TOKEN, API_KEY, CLIENT_CODE, FEED_TOKEN,max_retry_attempt=2, retry_strategy=0, retry_delay=10, retry_duration=30)

    #retry_strategy=1 for exponential retry mechanism
    # sws = SmartWebSocketV2(AUTH_TOKEN, API_KEY, CLIENT_CODE, FEED_TOKEN,max_retry_attempt=3, retry_strategy=1, retry_delay=10,retry_multiplier=2, retry_duration=30)
on_data_message = []
def on_data(wsapp, message):
    on_data_message.append(message)
    logger.info("Ticks: {}".format(message))
        # close_connection()

def on_control_message(wsapp, message):
    logger.info(f"Control Message: {message}")

def on_open(wsapp):
    logger.info("on open")
    some_error_condition = False
    if some_error_condition:
        error_message = "Simulated error"
        if hasattr(wsapp, 'on_error'):
            wsapp.on_error("Custom Error Type", error_message)
    else:
        sws.subscribe(correlation_id, mode, token_list)
            # sws.unsubscribe(correlation_id, mode, token_list1)

def on_error(wsapp, error):
    logger.error(error)

def on_close(wsapp):
    logger.info("Close")

def close_connection():
     sws.close_connection()


    # Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error
sws.on_close = on_close
sws.on_control_message = on_control_message


sws.connect(), on_data_message



# 3

In [ ]:
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
from logzero import logger
from websocket import WebSocketApp
import io
import logging


AUTH_TOKEN = ""
API_KEY = ''
CLIENT_CODE = ''
FEED_TOKEN = ""
correlation_id = "abc123"
action = 1
mode = 1
token_list = [
    {
        "exchangeType": 3,
        "tokens": ["500034"]
    }
]



sws = SmartWebSocketV2(AUTH_TOKEN, API_KEY, CLIENT_CODE, FEED_TOKEN,max_retry_attempt=2, retry_strategy=0, retry_delay=10, retry_duration=30)

    #retry_strategy=1 for exponential retry mechanism
    # sws = SmartWebSocketV2(AUTH_TOKEN, API_KEY, CLIENT_CODE, FEED_TOKEN,max_retry_attempt=3, retry_strategy=1, retry_delay=10,retry_multiplier=2, retry_duration=30)

def on_data(wsapp, message):
    logger.info("Ticks: {}".format(message))
        # close_connection()

def on_control_message(wsapp, message):
    logger.info(f"Control Message: {message}")

def on_open(wsapp):
    logger.info("on open")
    some_error_condition = False
    if some_error_condition:
        error_message = "Simulated error"
        if hasattr(wsapp, 'on_error'):
            wsapp.on_error("Custom Error Type", error_message)
    else:
        sws.subscribe(correlation_id, mode, token_list)
            # sws.unsubscribe(correlation_id, mode, token_list1)

def on_error(wsapp, error):
    logger.error(error)

def on_close(wsapp):
    logger.info("Close")

def close_connection():
     sws.close_connection()


    # Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error
sws.on_close = on_close
sws.on_control_message = on_control_message


sws.connect()

